In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
import time

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from src import settings

ml_1m_root_path = settings.DATA_ROOT_PATH + "/site_data/ml-1m/"

In [2]:
ratings = pd.read_csv(ml_1m_root_path + 'raw_data/ratings.dat', delimiter='::', header=None, engine='python', names=['uid', 'itemid', 'rating', 'server_time'])
ratings.head()
# lay 5 gia tri tuong tac dau tien, moi tuong tac lay: uid, itemid, rating, server_time.

,uid,itemid,rating,server_time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
print(ratings.count())

uid            1000209
itemid         1000209
rating         1000209
server_time    1000209
dtype: int64


In [4]:
item_df = pd.read_csv(ml_1m_root_path + 'raw_data/movies.dat', delimiter='::', header=None, engine='python', names=['itemid', 'title', 'genres'])
item_df.head()

,itemid,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
new_list_item= item_df.loc[:,'itemid'].tolist()
print "Length list item:", len(new_list_item)

Length list item: 3883


In [6]:
print(ratings.count())

uid            1000209
itemid         1000209
rating         1000209
server_time    1000209
dtype: int64


In [7]:
# loai bo cac gia tri ratings trong bo du lieu ratings
# doi voi cac item_id co rating ma khong co trong danh sach cac item (dl sai).
new_rating = ratings[ratings.itemid.isin(new_list_item)]
new_rating.head()

,uid,itemid,rating,server_time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
with open(ml_1m_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}

print(len(active_item_dict))
# print (active_item_dict)
# item_df = item_df[item_df.itemid.isin(active_item_list)]
# item_df.count()

3706


In [9]:
genre_dict = {'Action': 0, 
              'Adventure': 1,
'Animation':2
, '''Children's''': 3
, 'Comedy': 4
, 'Crime': 5
, 'Documentary': 6
, 'Drama': 7
, 'Fantasy': 8
, 'Film-Noir': 9
, 'Horror': 10
, 'Musical': 11
, 'Mystery': 12
, 'Romance': 13
, 'Sci-Fi': 14
, 'Thriller': 15
, 'War': 16
, 'Western': 17}
genre_dict
# list category for list movie.

{'Action': 0,
 'Adventure': 1,
 'Animation': 2,
 "Children's": 3,
 'Comedy': 4,
 'Crime': 5,
 'Documentary': 6,
 'Drama': 7,
 'Fantasy': 8,
 'Film-Noir': 9,
 'Horror': 10,
 'Musical': 11,
 'Mystery': 12,
 'Romance': 13,
 'Sci-Fi': 14,
 'Thriller': 15,
 'War': 16,
 'Western': 17}

In [10]:
new_rating.head()

,uid,itemid,rating,server_time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
# apply len tat cac cac phan tu cua item_df voi ham vo danh
y = new_rating.apply(lambda x: [active_item_dict[x[0]], x[2]], axis=1, result_type='expand')
y.head()
# doi voi moi item trong item_df => thay id moi, categories
# 40,Animation|Children's|Comedy
# 573,Adventure|Children's|Fantasy
# 1333,Comedy|Romance
# 450,Comedy|Drama

KeyError: (51, u'occurred at index 7260')

In [15]:
# write data frame into csv file
y.to_csv(ml_1m_root_path + 'i2pcat.txt', header=False, index=False)

# Sorting ratings.txt by time tang dan (tu qua khu den hien tai)

In [2]:
ml_df = pd.read_csv(ml_1m_root_path + 'ratings.txt', header=None, names=['uid', 'itemid', 'time_implicit','number_im','time_explicit','number_ex'])
print(ml_df.head())

   uid  itemid  time_implicit  number_im  time_explicit  number_ex
0    0       0      978300760          1      978300760          1
1    0       1      978302109          1             -1          0
2    0       2      978301968          1             -1          0
3    0       3      978300275          1      978300275          1
4    0       4      978824291          1      978824291          1


In [3]:
print 'Sorting by uid and time tang dan (tu qua khu cho den bay gio):'
ml_df = ml_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time_implicit"], ascending = True)).reset_index(drop=True)
print (ml_df[:50])

Sorting by uid and time tang dan (tu qua khu cho den bay gio):
    uid  itemid  time_implicit  number_im  time_explicit  number_ex
0     0      31      978300019          1      978300019          1
1     0      27      978300055          1      978300055          1
2     0      37      978300055          1      978300055          1
3     0      22      978300055          1      978300055          1
4     0      24      978300103          1             -1          0
5     0      36      978300172          1      978300172          1
6     0       3      978300275          1      978300275          1
7     0      47      978300719          1      978300719          1
8     0       7      978300719          1      978300719          1
9     0      21      978300760          1             -1          0
10    0       0      978300760          1      978300760          1
11    0      44      978300760          1      978300760          1
12    0       9      978301368          1      978301

In [5]:
# in ra file 
ml_df.to_csv(ml_1m_root_path + 'ratings_sort_tuc.txt', index=False, header=False)

# New task preprocessing for retail rocket dataset.

In [34]:
# events_df = pd.read_csv(retail_rocket_root_path + 'raw_data/events.csv')
# category_tree_df = pd.read_csv(retail_rocket_root_path + 'raw_data/category_tree.csv')

# item_properties file 
# 417 053 unique items
# every row in the file has corresponding timestamp
# item properties file contain timestamp column because all of them are time dependent
# since properties may change over time, e.g. price, category, etc

# value of categoryid properties contain item category identifier
# value of available properties contains availability of the item ( 1: available, 0: ow)
# All numerical values were marked with "n" char at the beginning, and have 3 digits precision after decimal point


item_properties_1_df = pd.read_csv(retail_rocket_root_path + 'raw_data/item_properties_part1.csv')
item_properties_2_df = pd.read_csv(retail_rocket_root_path + 'raw_data/item_properties_part2.csv')
# timestamp,itemid,property,value

# 1435460400000,460429,categoryid,1338
# 1441508400000,206783,888,1116713 960601 n277.200
# 1439089200000,395014,400,n552.000 639502 n720.000 424566
# 1431226800000,59481,790,n15360.000
# 1431831600000,156781,917,828513
# 1436065200000,285026,available,0
# 1434250800000,89534,213,1121373
# 1431831600000,264312,6,319724
# 1433646000000,229370,202,1330310
# 1434250800000,98113,451,1141052 n48.000
# 1439089200000,450113,888,1038400 45956 n504.000

In [35]:
item_properties_1_df[:11]

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
5,1436065200000,285026,available,0
6,1434250800000,89534,213,1121373
7,1431831600000,264312,6,319724
8,1433646000000,229370,202,1330310
9,1434250800000,98113,451,1141052 n48.000


In [17]:
item_properties_2_df[:11]

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062
5,1433041200000,286407,202,820407
6,1438484400000,256368,888,437265 1296497 n24.000 229949 651738 285933
7,1437879600000,307534,888,150169 212349 1095303 824508 1257235 153900
8,1439089200000,102767,888,5135 790941 1055803 221748 122132 n12.000 1135...
9,1431831600000,215180,71,1096621


In [36]:
events_df = pd.read_csv(retail_rocket_root_path + 'raw_data/events.csv')
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [37]:
# loc function: access a group of row and columns by labels.

# item_properties_1_df.property: dua ra item_id va properties cua item_properties_1_df
# 0           categoryid
# 1                  888
# 2                  400
# 3                  790
# 4                  917

# item_properties_1_df.property == 'categoryid'
#  0            True
#  1           False
#  2           False
#  3           False
#  4           False
#  5           False
#  6           False

# item_properties_1_df.loc[item_properties_1_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]

# lấy ở item_properties_1_df voi property = categoryid voi cac gia tri tren cac cot timestam, itemid va value

# giam gia tri tu 10999999 xuong con 426305 ban ghi

# Value of the "categoryid" property contains item category identifier.

item_cat_1 = item_properties_1_df.loc[item_properties_1_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]
item_cat_1.head()

,timestamp,itemid,value
0,1435460400000,460429,1338
140,1432436400000,281245,1277
151,1435460400000,35575,1059
189,1437274800000,8313,1147
197,1437879600000,55102,47


In [39]:
item_cat_2 = item_properties_2_df.loc[item_properties_2_df.property == 'categoryid', ['timestamp', 'itemid', 'value']]
item = item_cat_1.append(item_cat_2)
# y = item.drop_duplicates(['itemid', 'value']).groupby(['itemid', 'value']).count()
# y.head()
# # temp = item.loc[item.itemid == 25]
# # temp.sort_values(by=['timestamp'])
# # y = item.groupby(['itemid', 'value']).count()
# y.loc[y.timestamp == 1]
print "Record retrive from 1:", len(item_cat_1)
print "Record retrive from 1:", len(item_cat_2)
print "Total records:", len(item)

Record retrive from 1: 426305
Record retrive from 1: 361909
Total records: 788214


In [27]:
item[:10]

,timestamp,itemid,value
0,1435460400000,460429,1338
140,1432436400000,281245,1277
151,1435460400000,35575,1059
189,1437274800000,8313,1147
197,1437879600000,55102,47
213,1433041200000,397079,619
237,1436670000000,265036,1228
254,1437879600000,124459,1277
310,1437879600000,350508,546
325,1439089200000,221365,1226


In [40]:
# y.loc[:, ['itemid', 'value']] # type: pd.DataFrame

# xoa cac ban ghi trung nhau tren cac cot item_id va value, va loc lai lay cac gia tri tren 2 cot itemid va value.
z = item.drop_duplicates(['itemid', 'value']).loc[:, ['itemid', 'value']]
print z.head()
print "\n"
print "Length of data after remove duplicate:", len(z)
# y.to_dict()
# z.to_csv(retail_rocket_root_path + 'temp.txt', index=False, header=False)

     itemid value
0    460429  1338
140  281245  1277
151   35575  1059
189    8313  1147
197   55102    47


Length of data after remove duplicate: 442432


In [41]:
# convert i2index.txt into dictionary
with open(retail_rocket_root_path + 'i2index.txt') as f: 
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}

# get list key from this dict.
active_item_list = active_item_dict.keys()

In [45]:
# remove cac phan tu trong z khong nam trong active item list (cac key lay tu file i2index.txt)
s = z[z.itemid.isin(active_item_list)]
# khong quan tam den value.
print 's head'
print s.head()

y = s.loc[:, 'itemid'].drop_duplicates() # xoa item_id trung nhau va loc tren cot itemid
print "y data type:",type(y)
print (y.head())
print len(y)


s head
     itemid value
189    8313  1147
408   96660   498
504  244437   438
804  313481  1613
836  366108   498
y data type: <class 'pandas.core.series.Series'>
189      8313
408     96660
504    244437
804    313481
836    366108
Name: itemid, dtype: int64
83274


In [46]:
# apply len tat ca cac phan tu cua s voi ham vo danh lambda.
# s.head()
#      itemid value
# 189    8313  1147
# 408   96660   498
# 504  244437   438
# 804  313481  1613
# 836  366108   498

# active item dict
# 393218: 27821,
#  393219: 30596,
#  393220: 2768,
#  152918: 36242,
#  6: 5054,

# thay the cac phan tu tung hang trong s tuong ung boi active_item_dict[x[0]], x[1]
# t moi thu duoc bao gom key cua active_item_dict va value cua s tuong ung.
t = s.apply(lambda x: [active_item_dict[x[0]], x[1]], axis=1, result_type='expand')
t.head()
# s.to_csv(retail_rocket_root_path + 'i2pcat.txt', index=False, header=False)

,0,1
189,75509,1147
408,42912,498
504,13362,438
804,33257,1613
836,60227,498


In [18]:
t.to_csv(retail_rocket_root_path + 'i2pcat.txt', index=False, header=False)
# i2pcat: item_id va categories tuong ung cua item do.
# voi item_id la id cu cua item khi chua mapping

In [33]:
print y.head() # y bao gom id cua ban ghi va tuong ung id item.
x = y.to_list() # lay cac item_id trong y
print len(x)
# print x[:5]

189      8313
408     96660
504    244437
804    313481
836    366108
Name: itemid, dtype: int64
83274


# After that it change to movieLens explicit data

In [21]:
explicit_movielens_df = pd.read_csv(ml_1m_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time', 'rating'])
print(explicit_movielens_df.head())
# active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|', skiprows=1)
# active_item_df.head()

   uid  itemid       time  rating
0    1    1193  978300760       5
1    1    3408  978300275       4
2    1    2355  978824291       5
3    1    1287  978302039       5
4    1    2804  978300719       5


In [23]:
# load list key for i2index
with open(ml_1m_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print(len(active_item_list))

3706


In [24]:
# list key index from u2index.
with open(ml_1m_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))

6040


In [25]:
# loai bo tuong tac ma user khong co trong active_user_list
explicit_movielens_df = explicit_movielens_df[explicit_movielens_df.uid.isin(active_user_list)]
explicit_movielens_df = explicit_movielens_df[explicit_movielens_df.itemid.isin(active_item_list)]

print 'explicit movielens df before:'
print (explicit_movielens_df.head())

explicit movielens df before:
   uid  itemid       time  rating
0    1    1193  978300760       5
1    1    3408  978300275       4
2    1    2355  978824291       5
3    1    1287  978302039       5
4    1    2804  978300719       5


In [27]:
print 'Sorting by uid and time giam dan (cang xa so voi hien tai):'
# a = retail_rocket_df.sort_values(by=['uid'])
explicit_movielens_df = explicit_movielens_df.groupby(["uid"]).apply(lambda x: x.sort_values(["time"], ascending = False)).reset_index(drop=True)
print (explicit_movielens_df[:50])

Sorting by uid and time giam dan (cang xa so voi hien tai):
    uid  itemid       time  rating
0     1      48  978824351       5
1     1    1566  978824330       4
2     1    1907  978824330       4
3     1     783  978824291       4
4     1    2294  978824291       4
5     1    2355  978824291       5
6     1     595  978824268       5
7     1       1  978824268       5
8     1     588  978824268       4
9     1     527  978824195       5
10    1    1545  978824139       4
11    1    2398  978302281       4
12    1     594  978302268       4
13    1    1029  978302205       5
14    1    2791  978302188       4
15    1    3114  978302174       4
16    1     531  978302149       4
17    1    2918  978302124       4
18    1    2762  978302091       4
19    1    1246  978302091       4
20    1    1287  978302039       5
21    1    2797  978302039       4
22    1    1097  978301953       4
23    1    2018  978301777       4
24    1    1028  978301777       5
25    1     150  978301777    

In [28]:
# thay the rating ban dau boi user_id, item_id moi trong u2index va i2index.
explicit_movielens_df = explicit_movielens_df.apply(lambda x: [active_user_dict[x[0]], active_item_dict[x[1]]], axis=1, result_type='expand')
print(explicit_movielens_df.head())

   0   1
0  0  25
1  0  32
2  0  34
3  0  35
4  0  30


In [29]:
explicit_movielens_df.to_csv(ml_1m_root_path + 'ui_ex.txt', index=False, header=False)

# generate item representation.

In [9]:
# using item_repr.txt was made before to generate vector for this item.
active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|', skiprows=1)
print(active_item_df.head())

   itemid                                               repr
0    3506  [[17, 0.25], [80, 0.25], [219, 0.25], [464, 0....
1    2293  [[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0...
2    6555  [[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0....
3    8448  [[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]
4    1818  [[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]
/home/tucng/Desktop/RS/project/src/site_data/recobell/


In [15]:
# make a dictionary for each item 
# each of element is a key of item and value of categories
item_repr_dict = {}
for _, row in active_item_df.iterrows():
    item_repr_dict[row[0]] = row[1]

print(item_repr_dict[0])
print(item_repr_dict[3506])

[[14, 0.25], [73, 0.25], [426, 0.25], [1620, 0.25]]
[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0.25]]


In [17]:
explicit_recobell_df['num'] = 1

In [19]:
explicit_recobell_df.columns = ['uid', 'itemid', 'num']
explicit_recobell_df.head()

,uid,itemid,num
0,0,0,1
1,0,3,1
2,0,4,1
3,0,6,1
4,0,7,1


In [22]:
import json

def json_string_to_dense_vector(string_vector, dimensions):
    """
    Chuyen vector thua dang string: [[9, 0.010176822], [118, 0.010578092], [264, 0.020403702]]
    ve vector thuong K chieu cua numpy
    """
    K = dimensions

    vector = np.zeros(K, dtype=np.float32)
    if string_vector is None or len(string_vector) < 1:
        return vector

    x = json.loads(string_vector)
    
    for element in x:
        vector[element[0]] = element[1]
    return vector

In [23]:
def dense_vector_to_list_sparse_vector(vector):
    """
    Chuyen vector np ve vector thua dang string: [[9, 0.010176822], [118, 0.010578092], [264, 0.020403702]]
    """
    K = len(vector)
    threshold = 1e-6

    list_temp = []
    for i in range(0, K):
        if vector[i] > threshold:
            list_temp.append([i, float(vector[i])])
    return list_temp

In [24]:
def list_sparse_vector_to_json_string(sv):
    # type: (list) -> str
    """Convert vector in the form list of (int, float) to string

    Parameters
    ----------
    sv : list of (int, float)
        sparse_vector to be converted

    Examples
    ---------
    >>> sparse_vector = [(1, 0.123232), (2, 5.34234234)]
    >>> list_sparse_vector_to_json_string(sparse_vector)
    '[[1, 0.123232], [2, 5.34234234]]'
    """

    sum = 0
    list_sparse = []

    for item in sv:
        sum += item[1]

    for item in sv:
        list_sparse.append([int(item[0]), float(item[1] / sum)])

    return json.dumps(list_sparse)


In [ ]:
def add_avg()

In [27]:
explicit_recobell_df.head()

,uid,itemid,num
0,0,0,1
1,0,3,1
2,0,4,1
3,0,6,1
4,0,7,1


In [25]:
# apply into each row of explicit_recobell_df
t = explicit_recobell_df.apply(lambda x: [x[0], json_string_to_dense_vector([x[1]], 1939), x[2]], axis=1, result_type='expand').groupby(by=['0'])
print(t.head())

TypeError: ('expected string or buffer', u'occurred at index 0')

In [22]:
# item_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/site_product.csv000', header=None, names=['itemid', 'price', 'cat1', 'cat2', 'cat3', 'cat4', 'brandid'])
# view_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/tiny_site_view_log.csv000', header=None, names=['server_time', 'device', 'session_id', 'uid', 'itemid'])
order_recobell_df = pd.read_csv(recobell_root_path + 'raw_data/tiny_site_order_log.csv000', header=None, names=['server_time', 'device', 'session_id', 'uid', 'itemid', 'order_id', 'quantity'])
view_recobell_df.head()
# preprocessing with recobell datasets

,server_time,device,session_id,uid,itemid
0,2016-08-13 12:38:47.512,MA,bgF9eb8ynC,b15f9e5,d9886b1
1,2016-08-14 13:41:34.30,MA,yzbfOf5b4e,8007201,e295b87
2,2016-08-14 13:37:57.682,MA,yzbfOf5b4e,8007201,cedc044
3,2016-08-11 19:04:41.939,MA,YBpaGQxzg3,99055ba,4f3ecec
4,2016-08-11 18:57:11.63,MA,YBpaGQxzg3,99055ba,ee3f827


In [3]:
item_recobell_df.nunique()

itemid     422516
price       24607
cat1           18
cat2           80
cat3          342
cat4         1499
brandid     10584
dtype: int64

In [18]:
active_item_df = pd.read_csv(recobell_root_path + 'item_repr.txt', header=None, names=['itemid', 'repr'], delimiter=',', quotechar='|')
active_item_df.head()

,itemid,repr
0,5919d61,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,1073c34,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,559fc56,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,67da1f9,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,b5eaeaa,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [33]:
z = active_item_df[active_item_df.itemid.isin(active_item_list)]
z = z.drop_duplicates(['itemid'])
z.count()

itemid    118293
repr      118293
dtype: int64

In [34]:
t = z.apply(lambda x: [active_item_dict[x[0]], x[1]], axis=1, result_type='expand')
t.head()

,0,1
0,3506,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,2293,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,6555,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,8448,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,1818,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [37]:
import csv
t.to_csv(recobell_root_path + 'item_repr.txt', index=False, header=False, quotechar='|', quoting=csv.QUOTE_ALL)

In [28]:
y = active_item_df.loc[:, 'itemid'].drop_duplicates()
y.count()

422516

In [29]:
x = y.tolist()
print(len(x))

422516


In [30]:
new_view_df = view_recobell_df[view_recobell_df.itemid.isin(x)]
new_order_df = order_recobell_df[order_recobell_df.itemid.isin(x)]
new_view_df.count()

server_time    4521505
device         4521505
session_id     4521505
uid            4521505
itemid         4521505
dtype: int64

In [31]:
new_view_df.to_csv(recobell_root_path + 'raw_data/new_tiny_site_view_log.csv000', index=False, header=False)
new_order_df.to_csv(recobell_root_path + 'raw_data/new_tiny_site_order_log.csv000', index=False, header=False)

In [27]:
z.head()

,itemid,repr
0,5919d61,"[[17, 0.25], [80, 0.25], [219, 0.25], [464, 0...."
1,1073c34,"[[16, 0.25], [46, 0.25], [335, 0.25], [1892, 0..."
2,559fc56,"[[9, 0.25], [83, 0.25], [153, 0.25], [1854, 0...."
3,67da1f9,"[[6, 0.25], [82, 0.25], [249, 0.25], [832, 0.25]]"
4,b5eaeaa,"[[1, 0.25], [38, 0.25], [286, 0.25], [841, 0.25]]"


In [ ]:
z.to_csv(recobell_root_path + 'item_repr.txt', index=False, header=False)

In [61]:
a =  item_recobell_df.loc[:, ['cat2', 'cat3']]
x =  item_recobell_df.loc[:, ['cat2']]

In [63]:
y = a.drop_duplicates().groupby(['cat3']).count()
y.loc[y.cat2 > 1]

,cat2
cat3,
0ea22c8,2
142a302,2
23bebb0,2
3919b77,2
44650a9,7
512f480,2
58961c3,2
5b889c3,2
7efeb2d,3


In [59]:
item_recobell_df[item_recobell_df.cat2 == 'bd0b0a5'].drop_duplicates(['cat1', 'cat2'])

,itemid,price,cat1,cat2,cat3,cat4,brandid
9,eb32384,67410,d9d0ba9,bd0b0a5,f1c99bd,af4f9f8,7a9f4fc
114195,257677b,39000,66dd73d,bd0b0a5,58961c3,a457db3,fd23080


In [38]:
b = set(item_recobell_df.cat1) | set(item_recobell_df.cat2) | set(item_recobell_df.cat3) | set(item_recobell_df.cat4)

In [39]:
len(b)

1838

In [10]:
c = list(set(item_recobell_df.cat1))

In [1]:
len(c)
c

NameError: name 'c' is not defined

# Pre-process for movielens data

In [3]:
ratings.head()

,uid,itemid,rating,server_time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
count_rating = ratings.groupby(['uid'])[['rating']].count()
count_rating[count_rating['rating'] < 20]

,rating
uid,


In [5]:
mean_rating = ratings.groupby(['uid']).agg({'rating': 'mean'})
mean_rating['uid'] = mean_rating.index
mean_rating[mean_rating['rating'] < 2]

# ratings[ratings['uid'] == 2744]

,rating,uid
uid,,
2744,1.304348,2744
3598,1.015385,3598
4349,1.962963,4349
4486,1.058824,4486
4539,1.815126,4539
5334,1.927273,5334
5850,1.844828,5850


In [7]:
count_rating['rating'].max()

2314

In [8]:
count_rating['rating'].min()

20

In [48]:
mean_rating
with open(ml_1m_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))
new_mean_rating = mean_rating.apply(lambda x: pd.Series([active_user_dict[x['uid']], x['rating']], index=['uid', 'rating']), axis=1)

6040


In [55]:
new_mean_rating['uid'] = new_mean_rating['uid'].apply(lambda x: int(x))

In [6]:
mean_rating.to_csv(ml_1m_root_path + 'user_mean_rating.txt', header=False, index=False, quotechar='|', quoting=csv.QUOTE_ALL)